##Bert Bi-GRU for sentiment analysis

In [ ]:
import torch #pytorch library

import random
import numpy as np

SEED = 1234

random.seed(SEED) #setting random seed
np.random.seed(SEED) #setting 
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 4.4MB/s 
     |████████████████████████████████| 1.1MB 23.8MB/s 
     |████████████████████████████████| 3.0MB 31.0MB/s 
     |████████████████████████████████| 890kB 58.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5581a4e5b1410261cec5025b100a34d2883476ac4a3bde9d9eb2412659dc47ee
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer
#loading the pre-trained bert-base-uncased tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#checking number of tokens in BERT vocabulary
len(tokenizer.vocab)

30522

In [ ]:
#tokenizing using bert tokenizer
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [ ]:
#numericalizing tokens
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [ ]:
init_token = tokenizer.cls_token # first token of the sequence 
eos_token = tokenizer.sep_token # last token of a sequence 
pad_token = tokenizer.pad_token # token used for padding
unk_token = tokenizer.unk_token # unknown token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
# the indexes of the special tokens by converting them using the vocabulary
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

# or we can get these indexes by explicitly getting them from the tokenizer.
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
# the model was trained on sequences with a defined maximum length. 
# Getting this maximum length.
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
# making our own tokenizing function that tokenizes sentences and cuts them to a 
# size (max_input_length - 2) 
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
from torchtext import data
#creating preprocessing pipelines for text and label columns
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
#importing dataset
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
#splitting train_data to train data and valid data
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:33, 899kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 28.9MB/s]


In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
print(vars(train_data.examples[6]))

{'text': [2619, 2182, 2941, 4102, 2023, 3185, 1999, 2070, 3971, 2000, 2152, 11501, 1012, 2085, 2008, 2003, 1037, 2613, 7683, 999, 1045, 1005, 1049, 1037, 2502, 19643, 5470, 2164, 2070, 1997, 2010, 3772, 4395, 2021, 2672, 1996, 2069, 2711, 2040, 2071, 2031, 2209, 2023, 2112, 2052, 2031, 2042, 2123, 12226, 3215, 1012, 2034, 2125, 1010, 2004, 2619, 4197, 2041, 1010, 19643, 2074, 2987, 1005, 1056, 2031, 1996, 3857, 2005, 1037, 2530, 2919, 1011, 3124, 10587, 4783, 1012, 2002, 1005, 1055, 2074, 2205, 1005, 7263, 1005, 2012, 2023, 2391, 1999, 2010, 2166, 1012, 2672, 2002, 2001, 2055, 1996, 2168, 4578, 2004, 2360, 20075, 2063, 7104, 1010, 2021, 7104, 2018, 1037, 3492, 5024, 3857, 1012, 19643, 3310, 2408, 2004, 1996, 2502, 3331, 2210, 4845, 2040, 6343, 11276, 2000, 2202, 5667, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2466, 2003, 4895, 7076, 21649, 1998, 2428, 2025, 23411, 1012, 1045, 2123, 1005, 1056, 2215, 2000, 27594, 2009, 2021, 1045, 2228, 1996, 4566, 1998, 2129, 1996, 27

In [ ]:
# getting sentence from list of indexes
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['someone', 'here', 'actually', 'compared', 'this', 'movie', 'in', 'some', 'ways', 'to', 'high', 'noon', '.', 'now', 'that', 'is', 'a', 'real', 'stretch', '!', 'i', "'", 'm', 'a', 'big', 'sinatra', 'fan', 'including', 'some', 'of', 'his', 'acting', 'roles', 'but', 'maybe', 'the', 'only', 'person', 'who', 'could', 'have', 'played', 'this', 'part', 'would', 'have', 'been', 'don', 'knot', '##ts', '.', 'first', 'off', ',', 'as', 'someone', 'pointed', 'out', ',', 'sinatra', 'just', 'doesn', "'", 't', 'have', 'the', 'build', 'for', 'a', 'western', 'bad', '-', 'guy', 'wanna', '##be', '.', 'he', "'", 's', 'just', 'too', "'", 'slight', "'", 'at', 'this', 'point', 'in', 'his', 'life', '.', 'maybe', 'he', 'was', 'about', 'the', 'same', 'height', 'as', 'say', 'audi', '##e', 'murphy', ',', 'but', 'murphy', 'had', 'a', 'pretty', 'solid', 'build', '.', 'sinatra', 'comes', 'across', 'as', 'the', 'big', 'talking', 'little', 'kid', 'who', 'nobody', 'ought', 'to', 'take', 'seriously', '.', '<', 'br', '/'

In [ ]:
#building label vocabulary
LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fb6216c0268>, {'neg': 0, 'pos': 1})


In [ ]:
#creating iterator for train, valid and test data
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
from transformers import BertTokenizer, BertModel
#importing the pretrained bert model
bert = BertModel.from_pretrained('bert-base-uncased')

Building our model

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        #we use bert pretrained embeddings
        embedding_dim = bert.config.to_dict()['hidden_size']
        #defining our GRU layer 
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        #defining fully connected layer
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        #defining dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            # The transformer returns the embeddings for the whole sequence 
            # as well as a pooled output. We require the embeddings
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [ ]:
#defining hyperparameters
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
# creating the model instance
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
#getting the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [ ]:
# we set the requires_grad to false for all the bert parameters because they are pre trained
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
#this reduces our number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [ ]:
# checking the names of the trainable parameters, ensuring they make sense. 
# As we can see, they are all the parameters of the GRU (rnn) and the linear layer (out).
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [ ]:
#importing optim library to define our optimizer
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
#defining our loss function
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# putting model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

Training our model

In [ ]:
#defining our accuracy function
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    """ training our model"""    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    """ evaluating our model on validation dataset"""    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    """calculating how long a training/evaluation epoch takes"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
#training our model
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # saving the models which have better valid_loss    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7m 5s
	Train Loss: 0.495 | Train Acc: 75.14%
	 Val. Loss: 0.299 |  Val. Acc: 87.67%
Epoch: 02 | Epoch Time: 7m 5s
	Train Loss: 0.287 | Train Acc: 88.35%
	 Val. Loss: 0.245 |  Val. Acc: 90.21%
Epoch: 03 | Epoch Time: 7m 5s
	Train Loss: 0.235 | Train Acc: 90.58%
	 Val. Loss: 0.227 |  Val. Acc: 90.90%
Epoch: 04 | Epoch Time: 7m 5s
	Train Loss: 0.219 | Train Acc: 91.46%
	 Val. Loss: 0.220 |  Val. Acc: 91.49%
Epoch: 05 | Epoch Time: 7m 5s
	Train Loss: 0.183 | Train Acc: 93.17%
	 Val. Loss: 0.232 |  Val. Acc: 90.89%
Epoch: 06 | Epoch Time: 7m 6s
	Train Loss: 0.162 | Train Acc: 93.80%
	 Val. Loss: 0.251 |  Val. Acc: 90.72%
Epoch: 07 | Epoch Time: 7m 6s
	Train Loss: 0.131 | Train Acc: 95.13%
	 Val. Loss: 0.248 |  Val. Acc: 91.89%
Epoch: 08 | Epoch Time: 7m 7s
	Train Loss: 0.110 | Train Acc: 95.94%
	 Val. Loss: 0.268 |  Val. Acc: 91.37%
Epoch: 09 | Epoch Time: 7m 7s
	Train Loss: 0.083 | Train Acc: 97.01%
	 Val. Loss: 0.309 |  Val. Acc: 90.99%
Epoch: 10 | Epoch Time: 7m 7

In [ ]:
#loading the model with minimum valid_loss
model.load_state_dict(torch.load('tut6-model.pt'))
# testing on our test_data
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.213 | Test Acc: 91.78%


In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    """ returns the prediction of model on the input sentence using the model"""
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.015748243778944016

In [ ]:
predict_sentiment(model, tokenizer, "This film is great")

0.950927734375